In [1]:
!curl -O https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4073  100  4073    0     0  19212      0 --:--:-- --:--:-- --:--:-- 19303


In [2]:
!curl -O https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  642k  100  642k    0     0  2759k      0 --:--:-- --:--:-- --:--:-- 2759k


In [3]:
!pip install minsearch

In [4]:
import json
import minsearch

/workspaces/llm-zoomcamp/01-intro/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [5]:
with open('documents.json','rt') as f:
    doc_raw = json.load(f)

In [6]:
documents = []

for course_dict in doc_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [7]:
documents[3]

{'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
 'section': 'General course-related questions',
 'question': 'Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?',
 'course': 'data-engineering-zoomcamp'}

In [8]:
index = minsearch.Index(
    text_fields=["question","text","section"],
    keyword_fields=["course"]
)

In [9]:
q = "The course has already started ,can I still enroll?"

In [10]:
index.fit(documents)

In [11]:
import os
from dotenv import load_dotenv

load_dotenv()

# Set OPENAI_API_KEY for libraries that look for it
api_key = os.getenv("API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

In [12]:
from openai import OpenAI

client = OpenAI()

In [13]:
q

'The course has already started ,can I still enroll?'

In [14]:
def search(query):
    boost = {
    'question': 3.0,
    'section': 0.5
    }

    results = index.search(
        query=query,
        filter_dict= {'course':'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [15]:
def build_prompt(query,search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE.
    
    QUESTION : {question}
    
    CONTEXT : {context}
    
    """.strip()
    
    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query,context=context).strip()

    return prompt

In [16]:
def llm(prompt):
    response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role':'user','content':prompt}]
    )
    
    return response.choices[0].message.content

In [19]:
from elasticsearch import Elasticsearch

In [25]:
es_client = Elasticsearch('http://localhost:9200')

In [26]:
es_client.info()

ObjectApiResponse({'name': 'a534715a6e8e', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'ftxcs0B1SESdcy1Q7L0FZA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [28]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [29]:
from tqdm.auto import tqdm

In [30]:
for doc in tqdm(documents):
    es_client.index(index=index_name,document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [31]:
q

'The course has already started ,can I still enroll?'

In [45]:
def elastic_search(query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": q,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source']) 

    return result_docs
    

In [48]:
def rag(query):
    search_results = elastic_search(q)
    prompt = build_prompt(query,search_results)
    answer = llm(prompt)

    return answer

In [49]:
rag(q)

"Yes, even if the course has already started, you're still eligible to enroll and submit the homework. Just be aware of the deadlines for turning in the final projects."